In [264]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/max/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [265]:
wn = nltk.WordNetLemmatizer()
wn.lemmatize("founded")

'founded'

In [266]:
df = pd.read_csv("words.csv")

In [267]:
def split_pascal_case(input_string):
    words = re.findall(r'[A-Z][a-z]*', input_string)
    return ' '.join(words)

def replace_punctuation_with_space(input_string):
    return re.sub(r'[\.\!\-\_]', ' ', input_string)


def split_string(input_string, chunk_size):
    return [input_string[i:i+chunk_size] for i in range(0, len(input_string), chunk_size)]

def clean_text(input_string):
    no_symbols = replace_punctuation_with_space(input_string).lower()
    no_symbols = no_symbols.replace(" ", "")
    # tokenized = re.split("\W+", no_symbols.lower())
    tokenized = split_string(no_symbols, 3)
    return tokenized

In [268]:
# count_vect = CountVectorizer(ngram_range=(1,3), analyzer=clean_text)
count_vect = TfidfVectorizer(analyzer=clean_text)
vectorizer = count_vect.fit(df['text'])
X = vectorizer.transform(df['text'])

In [269]:
tokenized_df = pd.DataFrame(X.toarray(), columns=count_vect.get_feature_names())
tokenized_df.head()

/home/max/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,1,2,],aba,abr,aco,act,ad,add,ade,...,рал,рь,сен,т,тяб,уст,фев,ь,ябр,янв
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [270]:
from sklearn.preprocessing import LabelEncoder

y_labels = df['label'].drop_duplicates().to_list()
y_labels

['ref',
 'address',
 'housenumber',
 'street',
 'city',
 'state',
 'country',
 'postcode',
 'email',
 'month',
 'opening_hours',
 'phone',
 'lat',
 'lon',
 'location',
 'unknown']

In [271]:
l_encoder = LabelEncoder()
l_encoder.fit(y_labels)

y = l_encoder.transform(df['label'])
l_encoder.classes_

array(['address', 'city', 'country', 'email', 'housenumber', 'lat',
       'location', 'lon', 'month', 'opening_hours', 'phone', 'postcode',
       'ref', 'state', 'street', 'unknown'], dtype='<U13')

In [272]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [273]:
# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [274]:
# k = 1  # Количество соседей для классификации
# model = KNeighborsClassifier(n_neighbors=k)
# model = RandomForestClassifier()
model = DecisionTreeClassifier()
# model = CategoricalNB()

In [275]:
# Обучение модели
model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

In [276]:
# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")
f1 = f1_score(y_test, y_pred, average="weighted")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

Accuracy: 0.6666666666666666
Precision: 0.8176844806763285
Recall: 0.6666666666666666
F1: 0.692286556036556


/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/max/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [281]:
prediction = model.predict(vectorizer.transform(["work"]))
l_encoder.classes_[prediction[0]]

'opening_hours'

In [278]:
def predict(word):
    prediction = model.predict(vectorizer.transform([word]))
    return l_encoder.classes_[prediction[0]]

In [279]:
params = [
    "addr",
    "features",
    "location",
    "address",
    "addr_full",
    "state",
    "monday",
    "work",
    "content"
]
for row in params:
    print(f"{row}: {predict(row)}")

addr: address
features: location
location: location
address: address
addr_full: address
state: state
monday: opening_hours
work: opening_hours
content: phone


In [280]:
import pickle

with open('model_autoparse/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('model_autoparse/vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

with open('model_autoparse/label_encoder.pkl', 'wb') as label_encoder_file:
    pickle.dump(l_encoder, label_encoder_file)